In [16]:
import sys
sys.path.append('../src')

import model_training_multiclassificacao as mtm

In [17]:
import importlib
importlib.reload(mtm)

<module 'utils' from '/Users/patricia/Documents/code/python/behavior-detection/notebooks/../src/utils.py'>

In [19]:
mtm.load_data()
X_train, X_test, y_train, y_test = mtm.split_train_test_data()

Número total de alunos: 30
Número de alunos no conjunto de teste: 6
Tamanho do conjunto de treino: 4414
Tamanho do conjunto de teste: 1111
